In [ ]:
from copy import copy

#### Insertion Rules  
if maximum is surpassed:
1. Give to left sibling
2. Give to right sibling
3. Split

In [11]:
n = 25000000

In [25]:
nums = [n/2**x for x in range(24)]
for i in range(len(nums)-1):
    print(nums[i] - nums[i+1] > 5000)

True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False


In [1]:
class BPTree(object):
    def __init__(self, internal_capacity, leaf_capacity):
        self.root = InternalNode(internal_capacity)
    
    def insert(self, tup):
        self.root = self.root.insert_into_internal(tup)

SyntaxError: unexpected EOF while parsing (<ipython-input-1-e9c1a8d8350d>, line 7)

In [10]:
n = 5 // 2
print([1,2,3,4,5][n+1:])
print([1,2,3,4,5][:n+1])

[4, 5]
[1, 2, 3]


In [2]:
class InternalNode(object):
    def __init__(self, internal_capacity, leaf_capacity):
        self.capacity = internal_capacity
        self.leaf_capacity = leaf_capacity
        self.items = []
        self.children = []
        self.min = None
        self.lsibling = None
        self.rsibling = None
        
    def __repr__(self):
        s = ''
        s += 'Internal Node\n'
        s += f'Capacity: {self.capacity}\n'
        s += f'Items: {self.items}\n'
        s += f'Children: ['
        for child in self.children:
            c = ''.join(['    ' + x + '\n' for x in str(child).splitlines()])
            s += f'\n{c}'
        s = s.rstrip(',')
        s += ']'
        return s
        
    def has_space(self):
        return len(self.items) < self.capacity
    
    def over_capacity(self):
        return len(self.items) > self.capacity
        
    def split(self):
        new_right = InternalNode(self.capacity, self.leaf_capacity)
        promote_idx = self.capacity//2
        new_right.items = self.items[promote_idx+1:]
        new_right.children = self.children[promote_idx+1:]
        self.items = self.items[:promote_idx]
        self.children = self.children[:promote_idx+1]
        
        # change siblings
        old_right = self.rsibling
        self.rsibling = new_right
        new_right.lsibling = self
        new_right.rsibling = old_right
        if old_right:
            old_right.lsibling = new_right
        
        return self.items[promote_idx]
        
    def insert(self, tup):
        if len(self.children) == 0:
            self.children.append(LeafNode(self.leaf_capacity))
        if isinstance(self.children[0], LeafNode):
            self.insert_into_leaf(tup)
        elif isinstance(self.children[0], InternalNode):
            self.insert_into_internal(tup)
        else:
            raise Exception(f"Children of type: {type(self.children[0])}")
        
    def rotate_internal(self, node, child_idx):
        if self.try_give_left(node, child_idx):
            return
        elif self.try_give_right(node, child_idx):
            return
        else: # Split
            self.try_leaf_split(node, child_idx)
        
    def insert_into_internal(self, tup):
        # index of child internal node
        child_idx = len(self.items)
        for i, item in enumerate(self.items):
            if tup < item:
                child_idx = i
        node = self.children[child_idx]
        
        # insert into node
        node.insert(tup)
        
        if node.over_capacity():
            self.rotate_internal(node, child_idx)
        
    def update_items_leaf(self, i):
        # update internal node items[i] to be minimum of proper leaf node
        leaf = self.children[i+1]
        self.items[i] = leaf.items[0]
        
    def update_minimum_leaf(self):
        leaf = self.children[0]
        self.min = leaf.items[0]
        
    def try_give_left(self, leaf, child_idx):
        if leaf.lsibling and leaf.lsibling.has_space():
            leaf.give_left()
            if child_idx == 0: # gave left sibling internal node the tuple
                self.lsibling.update_items_leaf(len(self.lsibling.items)-1)
                self.update_minimum_leaf() # could speed up by just using leaf.items[0]
            else:
                self.update_items_leaf(child_idx - 1)
            return True
        return False
    
    def try_give_right(self, leaf, child_idx):
        if leaf.rsibling and leaf.rsibling.has_space():
            leaf.give_right()
            if child_idx == self.capacity: # gave right sibling internal node the tuple
                self.rsibling.update_minimum_leaf()
            else:
                self.update_items_leaf(child_idx)
                if child_idx == 0:
                    self.update_minimum_leaf()
            return True
        return False
        
    def try_leaf_split(self, leaf, child_idx):
        leaf.split()
        new_right = leaf.rsibling
        self.items.insert(child_idx, new_right.items[0])
        self.children.insert(child_idx + 1, new_right)
        
    def rotate_leaf(self, leaf, child_idx):
        if self.try_give_left(leaf, child_idx):
            return
        elif self.try_give_right(leaf, child_idx):
            return
        else: # Split
            self.try_leaf_split(leaf, child_idx)
        
    def insert_into_leaf(self, tup):
        # index of child leaf node
        child_idx = len(self.items)
        for i, item in enumerate(self.items):
            if tup < item:
                child_idx = i
        leaf = self.children[child_idx]
        
        # insert tuple into leaf node
        leaf.insert(tup)
        
        # fix leaf node if it is over capacity
        if leaf.over_capacity():
            self.rotate_leaf(leaf, child_idx)

In [3]:
class LeafNode(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.items = []
        self.lsibling = None
        self.rsibling = None

    def __repr__(self):
        s = ''
        s += 'Leaf Node\n'
        s += f'Capacity: {self.capacity}\n'
        s += f'Items: {self.items}\n'
        return s
    
    def has_space(self):
        return len(self.items) < self.capacity
    
    def over_capacity(self):
        return len(self.items) > self.capacity
    
    def insert(self, tup):
        self.items.append(tup)
        self.items.sort()
    
    def give_left(self):
        item = self.items[0]
        del self.items[0]
        self.lsibling.insert(item)
    
    def give_right(self):
        item = self.items[-1]
        del self.items[-1]
        self.rsibling.insert(item)
        
    def split(self):
        # split items
        new_right = LeafNode(self.capacity)
        new_right.items = self.items[self.capacity//2 + 1:]
        
        self.items = self.items[:self.capacity//2 + 1]
        
        # change siblings
        old_right = self.rsibling
        self.rsibling = new_right
        new_right.lsibling = self
        new_right.rsibling = old_right
        if old_right:
            old_right.lsibling = new_right

In [4]:
in1 = InternalNode(2, 3)

In [5]:
for i in range(5,20):
    in1.insert(i)

In [6]:
in1

Internal Node
Capacity: 2
Items: [8, 11, 14, 17]
Children: [
    Leaf Node
    Capacity: 3
    Items: [5, 6, 7]

    Leaf Node
    Capacity: 3
    Items: [8, 9, 10]

    Leaf Node
    Capacity: 3
    Items: [11, 12, 13]

    Leaf Node
    Capacity: 3
    Items: [14, 15, 16]

    Leaf Node
    Capacity: 3
    Items: [17, 18, 19]
]

In [ ]:
ln0 = LeafNode(3)
ln0.insert(10)

ln1 = LeafNode(3)
ln1.insert(20)

ln0.rsibling = ln1
ln1.lsibling = ln0

In [ ]:
ln0

In [ ]:
ln1

In [ ]:
in1 = InternalNode(2)
in1.children.append(ln0)
in1.children.append(ln1)
in1.items.append(20)

In [ ]:
in1.insert_into_leaf(2)
in1

In [ ]:
in1.insert_into_leaf(15)
in1.insert_into_leaf(16)
in1.insert_into_leaf(17)
in1

In [ ]:
in1.insert_into_leaf(14)
in1

In [ ]:
l = [16]

In [ ]:
l.insert(0, )

In [ ]:
m = 3 # elements in internal nodes
l = 4 # elements in leaf nodes

In [ ]:
ln1 = LeafNode(l)

for i in range(5):
    ln1.insert(i)

print(ln1)